### "configr" biblioteket kan lese TOML

In [ ]:
library('configr')

In [ ]:
conf <- read.config('config.toml')

In [ ]:
conf[["FEATURES"]][["log_kvalitet"]]

### Age ranges er da en liste av lister, med navn på "kategori", og verdier som skal kodes til disse

In [ ]:
age_ranges <- conf$FORMATS$POP$age_ranges
age_ranges

In [ ]:
# Eksempeldatasett

In [ ]:
inpath = paste(conf$PATHS$mock, "/population_2019_10000", sep="")
inpath

In [ ]:
# Les inn datasettet
df <- spark.read.path(inpath)
# Konverter fra spark-dataframe til R dataframe
df <- as.data.frame(df)

In [ ]:
#df

### Påfør kolonne for ny grupperingsvariabel på alder, utifra formatene i konfigurasjonsfilen

In [ ]:
for(key in names(age_ranges)){
    
    value <-age_ranges[key][[1]]
    print(key)
    
    # Om det er to verdier i listen så antar vi at de representerer en range av verdier
    if (length(value) == 2){
        print("Do stuff with two elements")
        df$age_group <- replace(df$age_group, 
                                sapply(df$age, 
                                       function(p) all(p >= value[[1]] 
                                                    & p <= value[[2]])
                                       ),
                                       key
                               )
    # Om det er en eller flere enn to verdier, så looper vi bare over alle enkeltvis og setter dem
    } else if (length(value) > 2 | length(value) == 1) {
        print("Do stuff with one or many elements")
        for (elem in value) {
           df$age_group <- replace(df$age_group, 
                                sapply(df$age, 
                                       function(p) all(p == elem)
                                       ),
                                       key
                               ) 
        }
        
    } else {
        print("very few elements?")
    }
    writeLines("\n")
}

### Se på en sample av datasettet

In [ ]:
library(dplyr)

In [ ]:
sample_n(df, 30)